<a href="https://colab.research.google.com/github/Sabokou/NLP/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/Sabokou/NLP/main/QuestionGenerator/dev_cleaned.csv

--2022-01-04 15:46:02--  https://raw.githubusercontent.com/Sabokou/NLP/main/QuestionGenerator/dev_cleaned.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5468410 (5.2M) [text/plain]
Saving to: ‘dev_cleaned.csv’

dev_cleaned.csv     100%[===================>]   5.21M  --.-KB/s    in 0.06s   

2022-01-04 15:46:03 (89.6 MB/s) - ‘dev_cleaned.csv’ saved [5468410/5468410]



In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from numpy import random

In [3]:
data = pd.read_csv('dev_cleaned.csv', sep="´")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the separator encoded in utf-8 is > 1 char long, and the 'c' engine does not support such separators; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [4]:
data = data.drop(columns=['Frage'])

In [5]:
data.head

<bound method NDFrame.head of                                       Kombiniertes Feld
0     <answer> France <context> The Normans (Norman:...
1     <answer> 10th and 11th centuries <context> The...
2     <answer> Denmark, Iceland and Norway <context>...
3     <answer> Rollo <context> The Normans (Norman: ...
4     <answer> 10th century <context> The Normans (N...
...                                                 ...
5923  <answer> kilogram-force <context> The pound-fo...
5924  <answer> kilopond <context> The pound-force ha...
5925  <answer> slug <context> The pound-force has a ...
5926  <answer> kip <context> The pound-force has a m...
5927  <answer> sthène <context> The pound-force has ...

[5928 rows x 1 columns]>

In [6]:
def get_answer(text):
  index_of_context = text.index('<context>')
  index_of_answer = text.index('<answer>')
  start_index = index_of_answer + 9
  end_index = index_of_context - 1
  answer_text = text[start_index : end_index]
  return(answer_text)

def get_context(text):
  index_of_context = text.index('<context>')
  start_index = index_of_context + 10
  context_text = text[start_index :]
  return(context_text)

In [7]:
data['answer'] = data.apply(lambda row : get_answer(row['Kombiniertes Feld']), axis=1) 
data['context'] = data.apply(lambda row : get_context(row['Kombiniertes Feld']), axis=1)
data = data.drop(columns=['Kombiniertes Feld'])

In [8]:
data.head

<bound method NDFrame.head of                            answer                                            context
0                          France  The Normans (Norman: Nourmands; French: Norman...
1         10th and 11th centuries  The Normans (Norman: Nourmands; French: Norman...
2     Denmark, Iceland and Norway  The Normans (Norman: Nourmands; French: Norman...
3                           Rollo  The Normans (Norman: Nourmands; French: Norman...
4                    10th century  The Normans (Norman: Nourmands; French: Norman...
...                           ...                                                ...
5923               kilogram-force  The pound-force has a metric counterpart, less...
5924                     kilopond  The pound-force has a metric counterpart, less...
5925                         slug  The pound-force has a metric counterpart, less...
5926                          kip  The pound-force has a metric counterpart, less...
5927                       sthène  

In [9]:
def pick_random_question(data):
    '''
    Chooses a random question from the data

    Input: df (pandas DataFrame), containing cards q&a pairs of the current topic selected

    Output: tuple, index (int) DataFrame index of question choosen,  row of DataFrame at index
    '''
    length=len(data)-1
    index=random.randint(0,length)
    return data['context'].iloc[index]

In [10]:
#pick_random_question(data)
correct_context ="In the modern industrialized world, construction usually involves the translation of designs into reality. A formal design team may be assembled to plan the physical proceedings, and to integrate those proceedings with the other parts. The design usually consists of drawings and specifications, usually prepared by a design team including Architect, civil engineers, mechanical engineers, electrical engineers, structural engineers, fire protection engineers, planning consultants, architectural consultants, and archaeological consultants. The design team is most commonly employed by (i.e. in contract with) the property owner. Under this system, once the design is completed by the design team, a number of construction companies or construction management companies may then be asked to make a bid for the work, either based directly on the design, or on the basis of drawings and a bill of quantities provided by a quantity surveyor. Following evaluation of bids, the owner typically awards a contract to the most cost efficient bidder."

In [11]:
def get_similarity(personal, correct):
  corpus = [personal, correct]
  vect = TfidfVectorizer(min_df=1, stop_words="english")
  tfidf = vect.fit_transform(corpus) 
  pairwise_similarity = tfidf * tfidf.T
  pairwise_similarity_arr = pairwise_similarity.toarray()
  score = pairwise_similarity_arr[0][1]
  print(score)

In [12]:
#personal_context = input("Context: ")
personal_context = "In the modern industrialized world, construction the translation of designs into reality. A formal may be assembled to plan the physical, and to integrate those proceedings with the other parts. The design usually consists of drawings and specifications, usually prepared by a design team including Architect, civil engineers, mechanical engineers, electrical engineers, structural engineers, fire protection engineers, planning consultants, architectural consultants, and archaeological consultants. The design team is most commonly employed by (i.e. in contract with) the property owner. Under this system, once the design is completed by the design team, a number of construction companies or construction management companies may then be asked to make a bid for the work, either based directly on the design, or on the basis of drawings and a bill of quantities provided by a quantity surveyor. Following evaluation of bids, awards a contract to the cost efficient bidder."

In [13]:
get_similarity(personal_context, correct_context)

0.979090922833454


In [14]:
#Innerhalb der WebApp ist die Funktion check_if_correct() unter dem Dokument "LearningForest.py" wiederzufinden, wo sie schließlich verwendet wird

def check_if_correct(correct_answer, my_answer):
  corpus = [my_answer, correct_answer]
  vect = TfidfVectorizer(min_df=1, stop_words="english")
  tfidf = vect.fit_transform(corpus) 
  pairwise_similarity = tfidf * tfidf.T
  pairwise_similarity_arr = pairwise_similarity.toarray()
  score = pairwise_similarity_arr[0][1]
  print(score)
  if score > 0.5:
    return True
    print("True")
  else:
    return False
    print("False")

In [15]:
check_if_correct("This is a Python file", "This is a Jupyter Notebook file")

0.2605556710562624


False